## Assignment 2 - Part 2 - Finding Twitter Influencers

### Top 100 Influencers

In [1]:
import pandas as pd
import tweepy

consumer_key = 'vZyOIzO8AZBgJ6U2DfjvyN9hi'
consumer_secret = 'jy8HKcp8OkQmacmyccnhaAq06e3BDBblas0f2ub0jWrRhVqan8'
access_token_key = '829374717941448704-eFTxdEdwJG67jNr8fI4vQEaCHSn7Css'
access_token_secret = '2kXFyEbXFhhi9bJNhlDpYHwak7798PhnU52Dx2YI2cak5'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
api=tweepy.API(auth)

#### Pull 5,000 Tweets from Twitter API

In [2]:
query = 'Trump'
count = 100
max_tweets = 5000
searched_tweets = []
last_id = -1
while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        new_tweets = api.search(q=query, count=count, max_id=str(last_id - 1))
        if not new_tweets:
            break
        searched_tweets.extend(new_tweets)
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # depending on TweepError.code, one may want to retry or wait
        # to keep things simple, we will give up on an error
        break

#### Mine Tweets for Relevant Information

In [7]:
trump_tweet_data = []

for tweet in searched_tweets: #Create a list of lists of the attributes of tweets
    k=[]
    k = k+[tweet.author.screen_name,tweet.author.listed_count,tweet.author.followers_count,tweet.text
           ,tweet.retweet_count,tweet.favorite_count,tweet.in_reply_to_screen_name]
    
    # If a retweet object exists, pull in user name of retweet. Otherwise, add blank entry.
    if hasattr(tweet, 'retweeted_status'):
        k.append(tweet.retweeted_status.user.screen_name)
    else: k.append('')
        
    # Pull in and convert Mentions object to a list of usernames
    mention_list = []
    for user_dict in tweet.entities['user_mentions']:
        if user_dict['screen_name'] != k[-1] and user_dict['screen_name'] != k[-2]: # Fix so that retweets and replies are not also included as mentions
            mention_list.append(user_dict['screen_name'])
            
    k.append(mention_list)
    
    trump_tweet_data.append(k)
print trump_tweet_data[:5] 

[[u'mmikrib', 89, 1621, u'RT @Breznican: BREAKING: Trump advisor Stephen Miller confirms it rubs the lotion on its skin or else it gets the hose again. https://t.co/\u2026', 5615, 0, None, u'Breznican', []], [u'ccrterminus', 44, 1616, u'RT @deIiveryboy: Simon looks like a trump supporter', 8, 0, None, u'deIiveryboy', []], [u'AngryLogician', 11, 208, u'RT @funder: US intelligence community operating under presumption Russia has "ears" on the Trump team #resist #trumpleaks #amjoy https://t.\u2026', 93, 0, None, u'funder', []], [u'1913Facts', 20, 589, u'/@EBONYMag Endless respect for THIS King kid but @ trump strategy: This blood sucker craves APPROVAL so negative attention is kryptonite!', 0, 0, u'EBONYMag', '', []], [u'Heleneafernande', 3, 123, u'RT @scambyarussian: Krispy Kream donuts yes\nPlease note that Justin received less of the popular vote than Trump,\n And has less support fro\u2026', 2, 0, None, u'scambyarussian', []]]


#### Convert to Dataframe

In [8]:
trump_tweet_data_df = pd.DataFrame(trump_tweet_data) #Convert it into a dataframe and publish a csv file
trump_tweet_data_df.columns = ['User','listed_count','followers_count','text','retweets',
                               'favorites','Reply_User','Retweet_User','Mention_Users']

trump_tweet_data_df.head()

,User,listed_count,followers_count,text,retweets,favorites,Reply_User,Retweet_User,Mention_Users
0,mmikrib,89,1621,RT @Breznican: BREAKING: Trump advisor Stephen...,5615,0,None,Breznican,[]
1,ccrterminus,44,1616,RT @deIiveryboy: Simon looks like a trump supp...,8,0,None,deIiveryboy,[]
2,AngryLogician,11,208,RT @funder: US intelligence community operatin...,93,0,None,funder,[]
3,1913Facts,20,589,/@EBONYMag Endless respect for THIS King kid b...,0,0,EBONYMag,,[]
4,Heleneafernande,3,123,RT @scambyarussian: Krispy Kream donuts yes\nP...,2,0,None,scambyarussian,[]


#### Define Attribute Weights

In [92]:
listed_count_weight = 0.44/(0.44+0.40+0.28)  #Calculate weights based on their importances
follower_count_weight = 0.40/(0.44+0.40+0.28)
retweet_count_weight = 0.28/(0.44+0.40+0.28)

#### Pre-processing

Remove unnecessary columns, keep only unique users, and normalize list, follower & retweet counts

In [111]:
from sklearn import preprocessing #Scale the data and score it

top_influencers_df = trump_tweet_data_df[['User','listed_count','followers_count','retweets']].drop_duplicates()

top_influencers_df['listed_norm'] = preprocessing.scale(top_influencers_df['listed_count'])
top_influencers_df['followers_norm'] = preprocessing.scale(top_influencers_df['followers_count'])
top_influencers_df['retweets_norm'] = preprocessing.scale(top_influencers_df['retweets'])

#### Calculated Weighted User Influence Score

In [116]:
top_influencers_df['Score'] = top_influencers_df['listed_norm']*listed_count_weight\
+top_influencers_df['followers_norm']*follower_count_weight\
+top_influencers_df['retweets_norm']*retweet_count_weight

top100influencers_trump = top_influencers_df.sort('Score',ascending=False)[:100]
top100influencers_trump[:10]

C:\Users\Dallas\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


,User,listed_count,followers_count,retweets,listed_norm,followers_norm,retweets_norm,Score
1170,htTweets,5951,5048757,1,11.757003,50.554688,-0.269788,22.606621
974,thehill,20015,1947355,4,39.959096,19.463365,-0.269404,22.582067
1495,24HorasTVN,7668,3092590,1,15.200048,30.944260,-0.269788,16.955522
446,darrenrovell,11472,1695674,26245,22.828089,16.940281,3.090865,15.790994
2872,AlbertoRavell,11353,1715397,2,22.589462,17.138003,-0.269660,14.927732
3733,teleSURtv,10989,1374561,0,21.859544,13.721148,-0.269916,13.420609
660,dailykos,7693,247580,3,15.250180,2.423247,-0.269532,6.789205
2245,LOLGOP,6891,322574,14,13.641955,3.175057,-0.268123,6.426257
528,FlorianDny,2,80,192595,-0.172338,-0.057922,24.392671,6.009777
2955,ChannelNewsAsia,4107,735461,1,8.059288,7.314218,-0.269788,5.710923


In [94]:
#trump_df_unique.sort('Score',ascending=False)[:100].to_csv('top100.csv',encoding='utf-8')

C:\Users\Dallas\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


## Influence Matrix

#### Create new DF with only relevant columns

##### Key Assumptions:
1. Their should be a reference line EACH TIME a user retweets another user (i.e. if a user retweets another user twice, their will be two entries)
2. Retweets and replies should not also count as mentions (i.e. if a user retweets a user, there will be an entry for just the retweet rathen then 1 for the retweet AND 1 for the mention).

In [26]:
influence_matrix_df = trump_tweet_data_df[['User','Reply_User','Retweet_User','Mention_Users']]
influence_matrix_df[:10]

,User,Reply_User,Retweet_User,Mention_Users
0,mmikrib,None,Breznican,[]
1,ccrterminus,None,deIiveryboy,[]
2,AngryLogician,None,funder,[]
3,1913Facts,EBONYMag,,[]
4,Heleneafernande,None,scambyarussian,[]
5,IBleedGandB,None,lorengrush,[]
6,JohnKollm,briangillespie,,[JoeNBC]
7,13Clarity,da13thsun,,[]
8,senthamarai34,None,virendersehwag,[]
9,beltrn_camila,None,,[]


In [29]:
influence_matrix = []

# Iterate over each of the rows creating a record (in the form of a list) for each connection
for index, row in influence_matrix_df.iterrows():
    influence_matrix.append([row['User'],row['User'],'Tweet']) # First row for the Tweet itself per instructions
    if row['Reply_User'] != None:
        influence_matrix.append([row['User'],row['Reply_User'],'Reply']) # Add Replies
    if len(row['Retweet_User']) > 0:
        influence_matrix.append([row['User'],row['Retweet_User'],'Retweet']) # Add Retweets
        influence_matrix.append([row['Retweet_User'],row['Retweet_User'],'Tweet']) # Add original tweet to list
    for user in row['Mention_Users']:
        influence_matrix.append([row['User'],user,'Mention']) # Add Mentions
        
# **NOTE: This currently assumes every Tweet that has been retweeted is not in the original list. 
# May need to bring into Tweet ID to avoid duplicates

In [34]:
influence_matrix_final = pd.DataFrame(influence_matrix) #Convert it into a dataframe and publish a csv file
influence_matrix_final.columns = ['User1', 'User2', 'Action']

influence_matrix_final.head()

,User1,User2,Action
0,mmikrib,mmikrib,Tweet
1,mmikrib,Breznican,Retweet
2,Breznican,Breznican,Tweet
3,ccrterminus,ccrterminus,Tweet
4,ccrterminus,deIiveryboy,Retweet


In [35]:
influence_matrix_final.to_csv('influence_matrix.csv',index=False)